In [46]:
#Hyperparameters to tune
#n_samples = 2000
n_features = 100
n_topics = 4
n_top_words = 20

moviesdf = pd.DataFrame(list(collection.find()))
movies = moviesdf.as_matrix()
moviestext = movies[:,1]
     
moviestext_train, moviestext_test = train_test_split(moviestext)
kf = KFold(moviestext_train.shape[0], n_folds=5)
vectorizer = TfidfVectorizer(max_features=n_features, stop_words='english')

for train_index, test_index in kf:    
    text_tfidf = vectorizer.fit_transform(list(\
            moviestext_train[train_index]))
    print text_tfidf.shape
    nmf = NMF(n_components=n_topics).fit(text_tfidf)
#     print nmf.reconstruction_err_

feature_names = vectorizer.get_feature_names()

for idx, sentiment in enumerate(nmf.components_):
    print "Sentiment #{}".format(idx)
    print(" ".join([feature_names[i]
                    for i in sentiment.argsort()[:-n_top_words - 1:-1]]))
    print

(1200, 100)
(1200, 100)
(1200, 100)
(1200, 100)
(1200, 100)
Sentiment #0
film characters films character really just scenes scene audience best plot good time way performance actually great like make people

Sentiment #1
movie movies good plot seen did like really people funny bad just original characters know thing character little audience actually

Sentiment #2
life story love man family world new young like home does john doesn time old director year role best comedy

Sentiment #3
action like hes bad theres doesnt just dont films good guy big effects movies time plot special better make script



In [44]:
n_topics = 4

def reconst_mse(target, left, right):
    return (np.array(target - left.dot(right))**2).mean()

def describe_nmf_results(document_term_mat, W, H, n_top_words = 15):
    print("Reconstruction error: %f") %(reconst_mse(document_term_mat, W, H))
    for topic_num, topic in enumerate(H):
        print("Topic %d:" % topic_num)
        print(" ".join([feature_words[i] \
                for i in topic.argsort()[:-n_top_words - 1:-1]]))
    return

moviesdf = pd.DataFrame(list(collection.find()))
movies = moviesdf.as_matrix()
moviestext = movies[:,1]
     
moviestext_train, moviestext_test = train_test_split(moviestext)
kf = KFold(moviestext_train.shape[0], n_folds=5)
vectorizer = TfidfVectorizer(max_features=n_features, stop_words='english')

for train_index, test_index in kf:    
    nmf = NMF(n_components=n_topics).fit(text_tfidf)
    text_tfidf = vectorizer.fit_transform(list(moviestext_train[train_index]))
    feature_words = vectorizer.get_feature_names()
    
    print("\n\n---------\nsklearn decomposition")
    nmf = NMF(n_components=n_topics)
    W_sklearn = nmf.fit_transform(text_tfidf)
    H_sklearn = nmf.components_
    describe_nmf_results(text_tfidf, W_sklearn, H_sklearn)



---------
sklearn decomposition
Reconstruction error: 0.006186
Topic 0:
film characters character films just really scenes scene audience performance good best great plot like
Topic 1:
movie movies seen good just plot funny characters like people did really know think bad
Topic 2:
action hes theres like bad doesnt dont just big effects good special films guy time
Topic 3:
life love story man new family world young does time doesn like john little role


---------
sklearn decomposition
Reconstruction error: 0.006247
Topic 0:
film characters films character scenes plot really audience just scene story like performance good seen
Topic 1:
movie movies funny good like just bad seen characters plot think comedy know really original
Topic 2:
life love man story world new family john like time young does little doesn people
Topic 3:
action theres doesnt hes bad big dont just effects like special guy good films better


---------
sklearn decomposition
Reconstruction error: 0.006207
Topic 0:
m

In [61]:
import numpy as np
import matplotlib.pyplot as plt
from code.util import Util
from pymongo import MongoClient
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import NMF
from mlxtend.classifier import EnsembleClassifier
%matplotlib inline

In [93]:
#who doesn't love utility classes            
util = Util()

#some classifiers that have been tuned
clf1 = LogisticRegression('l2')
clf2 = BernoulliNB(alpha=0,fit_prior=True)
#ensemble classifier
eclf = EnsembleClassifier(clfs=[clf1, clf2], weights=[1,1])

fig = plt.figure(1)

#get lists of scores and times
scores, times = util.TimevScore([clf1,clf2,eclf],review_sf,opinion_array,5,\
                       'f1_weighted',fig)

fig.plot(times,scores,'ro')
fig.xlabel('Seconds')
fig.ylabel('F1 Weighted Score')
for label, x, y in zip(['Logit','BernoulliNB','Ensemble'], times, scores):
    plt.annotate(
        label, 
        xy = (x, y), xytext = (40, 20),
        textcoords = 'offset points', ha = 'right', va = 'bottom',
        bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
fig.show()

#get lists of scores and times
scores, times = util.TimevScore([clf1,clf2,eclf],review_sf,opinion_array,5,\
                       'accuracy')

# plt.plot(times,scores,'ro')
# plt.xlabel('Seconds')
# plt.ylabel('F1 Weighted Score')
# for label, x, y in zip(['Logit','BernoulliNB','Ensemble'], times, scores):
#     plt.annotate(
#         label, 
#         xy = (x, y), xytext = (40, 20),
#         textcoords = 'offset points', ha = 'right', va = 'bottom',
#         bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
#         arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
# plt.show()

AttributeError: 'Figure' object has no attribute 'plot'

In [26]:
#data lists
review_list, opinion_list, sentiword_list, sentiment_list, word_list,\
            pos, neg = [],[],[],[],[],[],[]
#NMF hyper parameters
n_topics = 1
n_top_words = 1000

#PyMongo variables
client = MongoClient()
db = client['reviews']
collection = db['movies']
reviews = collection.find()

db1 = client['sentiment']
collection1 = db1['bingliu']
sentiments = collection1.find()

#build review and label lists
for review in reviews:
    opinion_list.append(review['Opinion'])
    review_list.append(review['Review'])

opinion_array = np.array(opinion_list)    

#build sentiment word, sentiment polarity, pos word, neg word
for sentiment in sentiments:
    sentiword_list.append(sentiment['Word'])
    sentiment_list.append(sentiment['Sentiment'])
    if sentiment['Sentiment'] == 1:
        pos.append(sentiment['Word'])
    elif sentiment['Sentiment'] == -1:
        neg.append(sentiment['Word'])


neg_vectorizer = TfidfVectorizer(decode_error='replace',strip_accents='unicode',\
                            vocabulary = neg, lowercase=True)
neg_tfidf = neg_vectorizer.fit_transform(review_list[0:1000])
pos_vectorizer = TfidfVectorizer(decode_error='replace',strip_accents='unicode',\
                            vocabulary = pos, lowercase=True)
pos_tfidf = pos_vectorizer.fit_transform(review_list[1000:2000])

neg_nmf = NMF(n_components=n_topics, random_state=1).fit(neg_tfidf)
pos_nmf = NMF(n_components=n_topics, random_state=1).fit(pos_tfidf)

neg_feature_names = neg_vectorizer.get_feature_names()
pos_feature_names = pos_vectorizer.get_feature_names()

for topic_idx, topic in enumerate(pos_nmf.components_):
    pos_nmf = [pos_feature_names[i]
                    for i in topic.argsort()[:-n_top_words - 1:-1]]

for topic_idx, topic in enumerate(neg_nmf.components_):
    neg_nmf = [neg_feature_names[i]
                    for i in topic.argsort()[:-n_top_words - 1:-1]]
    
nmfvectorizer = TfidfVectorizer(decode_error='replace',strip_accents='unicode',\
                    vocabulary = pos_nmf+neg_nmf, lowercase=True)
review_tfidf = nmfvectorizer.fit_transform(review_list)
review_sf = review_tfidf.copy()

#for every review
for i, review_s in enumerate(review_tfidf):
    #for every index (word) in the review
    for idx in review_tfidf[i].indices:
        if nmfvectorizer.vocabulary[idx] in neg_nmf:
            review_sf[i, idx] = review_tfidf[i, idx]*-1
        elif nmfvectorizer.vocabulary[idx] in pos_nmf:
            review_sf[i, idx] = review_tfidf[i, idx]
            
util = Util()

clf1 = LogisticRegression('l2')
clf2 = RandomForestClassifier(max_features='log2',n_estimators=100,\
                              criterion='entropy')
clf3 = BernoulliNB(alpha=0,fit_prior=True)
eclf = EnsembleClassifier(clfs=[clf1, clf2, clf3], weights=[1,1,1])

a, b = util.TimevScore([clf1,clf2,clf3,eclf],review_sf,opinion_array,5,\
                       'accuracy')
print a
print b

[ 0.838   0.831   0.6985  0.832 ]
[0.05299997329711914, 2.7219998836517334, 0.02200007438659668, 2.9790000915527344]


In [94]:
#NMF Hyperparameters
n_topics = 2
n_top_words = [10,20,50,100,1000]

#Lists of data
review_list, opinion_list, sentiword_list, sentiment_list,\
            pos, neg = [],[],[],[],[],[]
    
#PyMongo variables
client = MongoClient()
db = client['reviews']
collection = db['movies']
reviews = collection.find()

#PyMongo variables for sentiment
db1 = client['sentiment']
collection1 = db1['bingliu']
sentiments = collection1.find()

#build review and label lists
for review in reviews:
    opinion_list.append(review['Opinion'])
    review_list.append(review['Review'])

#label array
opinion_array = np.array(opinion_list)    

#build sentiment word, sentiment polarity, pos word, neg word
for sentiment in sentiments:
    sentiword_list.append(sentiment['Word'])

#vectorize words using sentiment words as vocab, lowercase, replace decode
#errors and strip accents using unicode
vectorizer = TfidfVectorizer(decode_error='replace',strip_accents='unicode',\
                            lowercase=True)
review_tfidf = vectorizer.fit_transform(review_list)

#NMF model fit using review_tfidf
nmf = NMF(n_components=n_topics, random_state=1).fit(review_tfidf)
#Feature names
feature_names = vectorizer.get_feature_names()

#loop through the topics in the nmf
for topic_idx, topic in enumerate(nmf.components_):
    #loop through the number of words in n_top_words
    for n_top_word in n_top_words:
        #build pos and neg word lists
        if topic_idx == 0:
            neg.append([feature_names[i]
                        for i in topic.argsort()[:-n_top_word - 1:-1]])
        elif topic_idx == 1:
            pos.append([feature_names[i]
                        for i in topic.argsort()[:-n_top_word - 1:-1]])

In [ ]:
#loop through the number of words in n_top_words
for j in xrange(len(n_top_words)):  
    #build a vectorizer using the NMF sets of words
    nmf_vectorizer = TfidfVectorizer(decode_error='replace',strip_accents='unicode',\
                                vocabulary = list(set(pos[j]+neg[j])), lowercase=True)
    nmf_review_tfidf = nmf_vectorizer.fit_transform(review_list)        
    #create a copy of the nmf_review_tfidf matrix to apply sentiment weights
    nmf_review_sf = nmf_review_tfidf.copy()

    #for every review
    for i, review_s in enumerate(nmf_review_tfidf):
        #for every index (word) in the review
        for idx in nmf_review_tfidf[i].indices:
            #apply sentiment weighting. if the word is in both lists then
            #the weight value is reduced to 0
            if nmf_vectorizer.vocabulary[idx] in neg and nmf_vectorizer.vocabulary[idx] in pos:
                nmf_review_sf[i, idx] = 0
            elif nmf_vectorizer.vocabulary[idx] in neg:
                nmf_review_sf[i, idx] = nmf_review_tfidf[i, idx]*-1
            elif nmf_vectorizer.vocabulary[idx] in pos:
                nmf_review_sf[i, idx] = nmf_review_tfidf[i, idx]
                
    util = Util()
    clf1 = LogisticRegression('l2')
    a, b = util.TimevScore([clf1],nmf_review_sf,opinion_array,5,\
                       'accuracy')
    print a
    print b
    print